In [14]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import warnings
import lightgbm as lgb
from workflow import main_indicators,Larry_simples_inverso,Larry_simples_compra,Larry_completo
from sklearn.model_selection import train_test_split
import plotly.express as px
from datetime import datetime
warnings.filterwarnings("ignore")

In [21]:
ativo = 'WIN@N'

if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

df = pd.DataFrame(mt5.copy_rates_from_pos(ativo, mt5.TIMEFRAME_M5, 0, 10000))
df.columns = ['Data','Open','High','Low','Close','Vol','','']
df['Data'] = pd.to_datetime(df['Data'], unit='s').apply(lambda x: str(x))

In [22]:
df

,Data,Open,High,Low,Close,Vol,,
0,2022-08-17 14:40:00,115475.0,115580.0,115365.0,115385.0,31077,5,103850
1,2022-08-17 14:45:00,115390.0,115550.0,115375.0,115545.0,24773,5,81736
2,2022-08-17 14:50:00,115545.0,115615.0,115515.0,115575.0,21925,5,73485
3,2022-08-17 14:55:00,115575.0,115755.0,115520.0,115690.0,34137,5,106536
4,2022-08-17 15:00:00,115690.0,116075.0,115535.0,115780.0,128776,5,386447
...,...,...,...,...,...,...,...,...
9995,2022-12-28 18:05:00,112005.0,112175.0,112000.0,112125.0,5137,0,54598
9996,2022-12-28 18:10:00,112125.0,112245.0,112085.0,112220.0,6597,0,67756
9997,2022-12-28 18:15:00,112220.0,112290.0,112110.0,112130.0,5477,0,52213
9998,2022-12-28 18:20:00,112125.0,112180.0,112100.0,112100.0,2359,0,17191


In [25]:
ativo = 'win_5_full'
colnames = ['Data','Open','High','Low','Close','Vol']
df = pd.read_csv(f'./Dados/{ativo}.csv', names = colnames, skiprows=1)

#df = df.iloc[:1000]

In [32]:
df = df[df['Data'] > '2022-10-01']

In [33]:
#df = main_indicators(df)

df_acao = Larry_completo(df, tempo_fechar=-10, excluir_seguidos = True, acao = 'all', x_stop = 1)

#df_acao = df_acao[df_acao['tend'] == 0]

compra = df_acao[df_acao['acao'] =='call']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='call'].shape[0]
venda = df_acao[df_acao['acao'] =='sell']['resultado_binario'].sum()/df_acao[df_acao['acao'] =='sell'].shape[0]

compra_result = df_acao[df_acao['acao'] =='call']['resultado_valor'].sum()
venda_result = df_acao[df_acao['acao'] =='sell']['resultado_valor'].sum()

print(f'Venda: {venda}% -- Compra: {compra}%')
print(f'Venda: {venda_result} -- Compra: {compra_result}')

Venda: 0.4065934065934066% -- Compra: 0.3739130434782609%
Venda: -1100.0 -- Compra: -4930.0


In [15]:
df_acao = df_acao.drop(columns = ['MA_9_diff_value', 'MA_9_tend', 'MA_9_tend_diff', 'preco','stop', 'realizacao_simplista', 'stop_real', 'tend'])

In [5]:
df_acao.columns

Index(['Data', 'Open', 'High', 'Low', 'Close', 'Vol', 'MA_5', 'MA_21', 'SAR',
       'BB_up', 'BB_mid', 'BB_low', 'OBV', 'macd', 'macdsignal', 'macdhist',
       'fastk', 'fastd', 'RSI', 'MOM', 'TRIX', 'CCI', 'ADX', 'dist_bb',
       'dist_media', 'dist_media_abs', 'dist_price_media', 'dist_RSI_70',
       'dist_RSI_30', 'MA_9', 'MA_100', 'acao', 'resultado_valor',
       'resultado_binario'],
      dtype='object')

In [55]:
fig = px.histogram(df_acao, x="dist_bb", color="resultado_valor_cluster")
fig.update_layout(barmode='overlay')
# Reduce opacity to see both histograms
fig.update_traces(opacity=1)
fig.show()

In [5]:
df_acao['resultado_binario'].sum()/df_acao.shape[0]

0.4847579199043634

In [48]:
df_acao['resultado_valor'].sum()

142420.0

In [59]:
df_new = df_acao[df_acao['dist_bb'] < 600]
df_new['resultado_binario'].sum()/df_new.shape[0]

0.4648466716529544

In [60]:
df_new['resultado_valor'].sum()

38340.0

In [27]:
a1 = df_acao["resultado_valor"].quantile(0.25)
a2 =df_acao["resultado_valor"].quantile(0.5)
a3 =df_acao["resultado_valor"].quantile(0.75)
a4 =df_acao["resultado_valor"].quantile(1)

In [28]:
df_acao["resultado_valor_cluster"] = df_acao["resultado_valor"].apply(lambda x: 1 if x < a1 else 2 if x < a2 else 3 if x < a3 else 4)

In [47]:
a3

225.0